In [1]:
# importing important libs
import pandas as pd
from statistics import mean, median

In [2]:
# print all columns - same name as in csv file
def list_columns(df):
    for column in list(df.columns):
        print(column)

In [3]:
# remove a list of columns
def remove_columns(df, columns):
    df = df.drop(columns=columns)
    return df

In [4]:
# remove rows by index
def remove_rows(df, indexes):
    df = df.drop(indexes)
    return df

In [5]:
# rename value in a list according to what is mapped
def rename_values(rename_mapping, values):
    renamed_values = []
    
    for value in values:
        renamed_values.append(rename_mapping.get(value, 'Not selected'))
    
    return renamed_values

In [6]:
# replace values in a column given a condition
def replace_values_by_condition(df, column, old_value, new_value, condition):
    if condition == 'eq':
        # replace values that respect the == condition
        df.loc[df[column] == old_value, column] = new_value
    elif condition == 'gt':
        # replace values that respect the > condition
        df.loc[df[column] > old_value, column] = new_value
    elif condition == 'lt':
        # replace values that respect the < condition
        df.loc[df[column] < old_value, column] = new_value
    elif condition == 'geq':
        # replace values that respect the >= condition
        df.loc[df[column] >= old_value, column] = new_value
    elif condition == 'leq':
        # replace values that respect the <= condition
        df.loc[df[column] <= old_value, column] = new_value
    elif condition == 'diff':
        # replace values that respect the != condition
        df.loc[df[column] != old_value, column] = new_value
    
    return df

In [7]:
# format dataframe with new columns names and removing what is not used
def format_df(df):
    # remove all columns specified in 'unused_columns.txt'
    with open('./unused_columns.txt', encoding="utf-8") as file_unused_columns:
        unused_columns = file_unused_columns.readlines()
    
    # remove '\n' character
    unused_columns = [unused_column.replace("\n", '') for unused_column in unused_columns]
    
    unnecessary_columns = unused_columns

    # drop all the columns which are not useful for us
    df = remove_columns(df, unnecessary_columns)
    # remove the first two rows - unnecessary data describing the columns
    df = remove_rows(df, [0, 1])
        
    # rename columns by those set in 'formatted_columns.txt'
    with open('./formatted_columns.txt', encoding="utf-8") as file_formatted_columns:
        formatted_columns = file_formatted_columns.readlines()
        
    # remove '\n' character
    formatted_columns = [formatted_column.replace("\n", '') for formatted_column in formatted_columns]
    
    df.columns = formatted_columns
    
    return df

In [8]:
def custom_groupby(df, groupby_column, count_columns, groupby_column_label, count_column_label, match_column_label, math):
    # what we want is the same of a groupby, but more generic and not assuming that the values grouped needs to be a number
    # we allow to groupby doing the mean, sum, median or simple count
    
    # get all values that appear in the groupby column
    groupby_column_values = list(df[groupby_column].unique())
    
    # dict to store each groupby column value and each to count column value
    group_to_count = {}
    for group in groupby_column_values:
        group_to_count[group] = {}
        for count in count_columns:
            # store as a list, because it give us flexibility to do several operations, from a simple len() to median of values
            group_to_count[group][count] = []
    
    # now we have something like: {'Business Analyst': {'Software Experience': [], 'ML Experience': []}, 'Data Scientist': ...}
    
    # for each row I build a dict storing for each value that will be row the value of each column
    for idx, row in df.iterrows():
        # what is the current grouped value from the chosen groupby_column
        current_grouped_value = row[groupby_column]
        # for the same current grouped value, we will increase with the value selected in the count column
        for count in count_columns:
            group_to_count[current_grouped_value][count].append(row[count])
    
    # now we have something like: {'Business Analyst': {'Software Experience': [5, 10, 0], 'ML Experience': [1, 6, 7]}, ...}
    
    # let's return a formatted df assuming
    grouped_df = pd.DataFrame()
    # what value was grouped
    grouped_values = []
    # what value exists for each group
    count_values = []
    # match of grouped value and its counted
    grouped_count_values = []
    
    for gtc in group_to_count:
        grouped_values = grouped_values + list(group_to_count[gtc].keys())
        count_values = count_values + [gtc for _ in range(len(group_to_count[gtc].keys()))]
        grouped_count = []
        
        for t in group_to_count[gtc]:
            if math == 'sum':
                grouped_count.append(sum(group_to_count[gtc][t]))
            if math == 'mean':
                grouped_count.append(mean(group_to_count[gtc][t]))
            if math == 'median':
                grouped_count.append(median(group_to_count[gtc][t]))
            if math == 'count':
                grouped_count.append(len(group_to_count[gtc][t]))
            
        grouped_count_values = grouped_count_values + grouped_count
        
    grouped_df[groupby_column_label] = grouped_values
    grouped_df[count_column_label] = count_values
    grouped_df[match_column_label] = grouped_count_values
    
    return grouped_df

In [9]:
# retrieve a formatted dataframe for the questions that we will work on
def get_df():
    # read survey csv
    df = pd.read_csv("data_project_851780_2022_05_02.csv", sep=";")
    df = format_df(df)
    return df

In [10]:
#remove all values equal to 'value' from 'col' in 'df'
def remove_value(df,value,col):
    if isinstance(value,str):
        df.drop(df.loc[df[col]==value].index, inplace=True)  
    else:
        if np.isnan(value) == True:
        #tratamento de nan
            print('nan')

In [11]:
#replace all values in received list to new_value
def replace_list_values_by_condition(df, column, old_values, new_value, condition):
    for value in old_values:
        replace_values_by_condition(df, column, value, new_value,'eq')

In [12]:
def format_wordcloud_text(texts, use_sep, stopwords=[]):
    # build a dict with a word and its importance/weight
    weighted_text = {}
    for text in texts:
        # split the text by tab
        if use_sep:
            splitted_text = text.split(" ")
            for split in splitted_text:
                if weighted_text.get(split):
                    weighted_text[split] = weighted_text[split] + 1
                else:
                    weighted_text[split] = 1
        else:
            # it will use the value as it is being read
            if weighted_text.get(text):
                weighted_text[text] = weighted_text[text] + 1
            else:
                weighted_text[text] = 1
                
    return weighted_text